In [ ]:
author="Jumabek Alikhanov"
date = 'Nov 19,2019'

### Download data directly from the link below and unzip it in the same directory

In [4]:
!gdown --id 1-t3RdDpmqMs4ABt9oobSapeNYTZJ9tpu
!unzip MachineLearningCSV.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-t3RdDpmqMs4ABt9oobSapeNYTZJ9tpu
To: /content/net_intrusion_detection/MachineLearningCSV.zip
100% 235M/235M [00:02<00:00, 102MB/s]


In [6]:
#imports
import os
from os.path import join
import glob
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
%load_ext autoreload

In [7]:
# load data
dataroot = 'MachineLearningCVE/'

In [8]:
from preprocessing import load_data
X,y = load_data(dataroot) # reads csv file and returns np array of X,y -> of shape (N,D) and (N,1)

MachineLearningCVE/*.csv
['MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', 'MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv', 'MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv', 'MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv', 'MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv', 'MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', 'MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv', 'MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv']
There are 2830743 flow records with 79 feature dimension
Data loaded.
Data preprocessing started...
Stripped column names
Dropped bad columns
There are 0 nan entries
Converted to numeric


In [13]:
from preprocessing import balance_data, normalize
#X = normalize(X)
SEED=2

In [15]:
# Random Foresty with 10 DecisionTrees
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics


K=5
results = {}
skf = StratifiedKFold(n_splits=K)

for fold_index, (dev_index,test_index) in enumerate(skf.split(X,y)): 
    print('---------------------------------------------')
    print('Fold #{}'.format(fold_index))    
    X_dev = X[dev_index]
    y_dev = y[dev_index]
    X_test = X[test_index]
    y_test = y[test_index]
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
    for train_index, val_index in sss.split(X_dev, y_dev): # runs only once
        X_train = X_dev[train_index]
        y_train = y_dev[train_index]
        X_val = X_dev[val_index]
        y_val = y_dev[val_index]
        break
    
    #X_train,y_train = balance_data(X_train,y_train,seed=SEED)
    clf = RandomForestClassifier(n_estimators=10, random_state=SEED)
    clf.fit(X_train,y_train)
    
    pred = clf.predict(X_val)
    imbalanced_val_acc = metrics.accuracy_score(y_val,pred)*100        
    val_acc = metrics.balanced_accuracy_score(y_val,pred)*100

    pred = clf.predict(X_test)
    test_acc = metrics.balanced_accuracy_score(y_test,pred)*100
    print('val acc:',imbalanced_val_acc)
    print('balanced val acc: ',val_acc)
    print('balanced test acc: ',test_acc)
    
    results[fold_index]= (imbalanced_val_acc, val_acc, test_acc)
    


---------------------------------------------
Fold #0
val acc: 99.85472015967535
balanced val acc:  84.57402498830538
balanced test acc:  80.69595867676547
---------------------------------------------
Fold #1
val acc: 99.82204323510385
balanced val acc:  82.32988792051918
balanced test acc:  82.9067562729627
---------------------------------------------
Fold #2
val acc: 99.84081038772938
balanced val acc:  83.02130895989173
balanced test acc:  81.4632409759166
---------------------------------------------
Fold #3
val acc: 99.91058003749015
balanced val acc:  78.5663213262709
balanced test acc:  81.73097334481285
---------------------------------------------
Fold #4
val acc: 99.85449936964447
balanced val acc:  86.23074730512053
balanced test acc:  73.63843326950139


In [16]:
sum_imbalanced_val_acc = 0
sum_val_acc = 0
sum_test_acc = 0
for fold_index,res in results.items():
    (acc,balanced_acc,test_acc) = res
    sum_imbalanced_val_acc+=acc
    sum_val_acc +=balanced_acc
    sum_test_acc+= test_acc
print("{}-fold evaluation:".format(K))
print('imbalanced val acc:',sum_imbalanced_val_acc/K)
print('val acc: ',sum_val_acc/K)
print('test acc: ',sum_test_acc/K)
print('{0:.2f} {1:.2f} {2:.2f}'.format(sum_imbalanced_val_acc/K,sum_val_acc/K,sum_test_acc/K))

5-fold evaluation:
imbalanced val acc: 99.85653063792864
val acc:  82.94445810002154
test acc:  80.0870725079918
99.86 82.94 80.09
